In [ ]:
import numpy as np


class Node:
    def __init__(self, nodes, weights, o_v=-1):
        self.value_output = o_v
        self.input_nodes = np.array(nodes)
        self.input_weights = np.array(weights)
        self.error = 0
        
    def activate(self, x):
        return 1/(1 + np.exp(-x))
    
    def S(self):
        vals = list(map(lambda x: x.value_output, self.input_nodes))
        return np.sum(vals * self.input_weights)
        
    def activate_deriv(self):
        return self.activate(self.S()) * (1 - self.activate(self.S()))
    
    def get_output(self):
        self.value_output = self.activate(self.S())
        

class Web:
    def __init__(self, x, cfg=[3, 2, 3]):
        self.config = cfg
        self.config.insert(0, len(x))
        self.lrs_num = len(self.config)
        self.lrs = []
        
        self.lrs.append([])
        for xn in x:
            self.lrs[0].append(Node([], [], xn))
        
        for i in range(1, self.lrs_num):
            self.lrs.append([])
            for j in range(self.config[i]):
                self.lrs[i].append(Node(self.lrs[i - 1], [(np.random.random() - 0.5) for i in range(self.config[i - 1])]))

    def get_probos(self):
        for lr in self.lrs[1:]:
            for nd in lr:
                nd.get_output()  
                
        return list(node.value_output for node in self.lrs[-1])
    
    def setx(self, x):
        for node, value in zip(self.lrs[0], x):
            node.value_output = value
        
    def learn(self, A, speed):
        for i in range(self.config[-1]):
            self.lrs[-1][i].error = self.lrs[-1][i].value_output - A[i]
        
        for node in self.lrs[self.lrs_num - 1]:
            for i in range(self.config[self.lrs_num - 2]):
                node.input_weights[i] -= speed * node.error * node.activate_deriv() * node.input_nodes[i].value_output
        
        if self.lrs_num > 1:
            for i in range(self.lrs_num - 2, 0, -1):
                for j in range(self.config[i]):
                    self.lrs[i][j].error = sum([x.error * x.activate_deriv() * x.input_weights[j] for x in self.lrs[i + 1]])

                for elem in self.lrs[i]:
                    for j in range(self.config[i - 1]):
                        elem.input_weights[j] -= speed * elem.error * elem.activate_deriv() * elem.input_nodes[j].value_output
